In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPClassifier, MLPRegressor

%matplotlib inline

import category_encoders as ce

In [69]:
df1 = pd.read_csv('.\\Washington Metropolitan Area Transit Authority\\wmata_train_predictions\\train_predictions.csv')

In [70]:
df2 = pd.read_csv('.\\Washington Metropolitan Area Transit Authority\\wmata_train_predictions\\train_arrivals.csv')

In [71]:
df1['PREDICTION_DTM'] = pd.to_datetime(df1['PREDICTION_DTM'], errors='ignore')
df1['Hour'] = df1['PREDICTION_DTM'].dt.hour
df1['Day'] = df1['PREDICTION_DTM'].dt.day
df1['New_ID'] = df1.UNIQUE_TRAIN_INDEX + df1.TRAIN_NUMBER + df1.STATION_ID + df1.PLATFORM_NUMBER.map(str) + df1.Hour.map(str) + df1.Day.map(str)

In [72]:
df2.ARRIVALTIME = pd.to_datetime(df2['ARRIVALTIME'], errors='ignore')
df2['Hour'] = df2['ARRIVALTIME'].dt.hour
df2['Day'] = df2['ARRIVALTIME'].dt.day
df2['New_ID'] = df2.UNIQUE_TRAIN_INDEX.map(str) + df2.TRAIN_NUMBER.map(str) + df2.STATION_ID + df2.PLATFORM_NUMBER.map(str) + df2.Hour.map(str) + df2.Day.map(str)

In [78]:
df1 = df1[df1.TRAIN_NUMBER != '0']
df1 = df1[df1.IS_FROM_SCHEDULE == 0]
df1 = df1[df1.IS_REVENUE == 1]

In [79]:
print(df1.shape)
print(df1.New_ID.unique().shape)

(6326371, 12)
(325348,)


In [80]:
print(df2.shape)
print(df2.New_ID.unique().shape)

(210187, 9)
(210187,)


In [81]:
df2.drop_duplicates(subset='New_ID', keep="first", inplace=True)

In [82]:
df1.STATION_ID.unique()

array(['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09',
       'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'B01', 'B02', 'B03',
       'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B10', 'B11', 'B35',
       'C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C09',
       'C10', 'C12', 'C13', 'C14', 'C15', 'D01', 'D02', 'D03', 'D04',
       'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D13',
       'E01', 'E02', 'E03', 'E04', 'E05', 'E06', 'E07', 'E08', 'E09',
       'E10', 'F01', 'F02', 'F03', 'F04', 'F05', 'F06', 'F07', 'F08',
       'F09', 'F10', 'F11', 'G01', 'G02', 'G03', 'G04', 'G05', 'J02',
       'J03', 'K01', 'K02', 'K03', 'K04', 'K05', 'K06', 'K07', 'K08',
       'N01', 'N02', 'N03', 'N04', 'N06'], dtype=object)

%%time
def merge_arrival_and_predictions(arrival, prediction):
    prediction['ARRIVALTIME'] = np.nan
    for index, row in arrival.iterrows():
        #mask = prediction[prediction.New_ID == row.New_ID]
        #prediction[mask]['ARRIVALTIME'] = row.ARRIVALTIME
        prediction.loc[prediction['New_ID'] == row.New_ID, 'ARRIVALTIME'] = row.ARRIVALTIME
    return prediction

x = merge_arrival_and_predictions(df2.head(100), df1.head(500))

In [83]:
df3 = df1.merge(df2, left_on='New_ID', right_on='New_ID')

In [84]:
df3.head()

,PREDICTION_DTM,STATION_ID_x,PLATFORM_NUMBER_x,LINE_CODE,IS_REVENUE,IS_FROM_SCHEDULE,UNIQUE_TRAIN_INDEX_x,TRAIN_NUMBER_x,ETA,Hour_x,Day_x,New_ID,PLATFORM,ARRIVALTIME,STATION_ID_y,PLATFORM_NUMBER_y,UNIQUE_TRAIN_INDEX_y,TRAIN_NUMBER_y,Hour_y,Day_y
0,2019-01-22 05:00:31,A01,1,RD,1,0,245,113,2061,5,22,245113A011522,A01-1,2019-01-22 05:35:48,A01,1,245,113,5,22
1,2019-01-22 05:01:32,A01,1,RD,1,0,245,113,1994,5,22,245113A011522,A01-1,2019-01-22 05:35:48,A01,1,245,113,5,22
2,2019-01-22 05:02:33,A01,1,RD,1,0,245,113,1926,5,22,245113A011522,A01-1,2019-01-22 05:35:48,A01,1,245,113,5,22
3,2019-01-22 05:03:34,A01,1,RD,1,0,245,113,1822,5,22,245113A011522,A01-1,2019-01-22 05:35:48,A01,1,245,113,5,22
4,2019-01-22 05:04:34,A01,1,RD,1,0,245,113,1804,5,22,245113A011522,A01-1,2019-01-22 05:35:48,A01,1,245,113,5,22


def merge_arrival_and_predictions(arrival, prediction):
    prediction['ARRIVALTIME'] = np.nan
    for index, row in arrival.iterrows():
        prediction['ARRIVALTIME'] = np.where(prediction['New_ID'] == row.New_ID, row.ARRIVALTIME, np.nan)

merge_arrival_and_predictions(df2, df1)

df1.to_csv('df1.csv', encoding='utf-8')
df2.to_csv('df2.csv', encoding='utf-8')

In [85]:
df4 = pd.DataFrame()
df4['New_ID'] = df3.New_ID
df4['ARRIVALTIME'] = df3.ARRIVALTIME
df4['PREDICTION_DTM'] = df3.PREDICTION_DTM
df4['ETA'] = df3.ETA

In [86]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3408204 entries, 0 to 3408203
Data columns (total 4 columns):
New_ID            object
ARRIVALTIME       datetime64[ns]
PREDICTION_DTM    datetime64[ns]
ETA               int64
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 130.0+ MB


def time_diff(pred, arr):
    x = (pred.hour*3600+pred.minute*60+pred.second)/60
    y = (arr.hour*3600+arr.minute*60+arr.second)/60
    return x - y

df4['diff'] = df4.apply(lambda row: time_diff(row.PREDICTION_DTM, row.ARRIVALTIME), axis=1)

In [87]:
df4['test'] = df4.PREDICTION_DTM + df4.ETA.astype("timedelta64[s]")

In [88]:
p_date = df4['test'].dt
a_date = df4['ARRIVALTIME'].dt
df4['pred_time'] = (p_date.hour*3600+p_date.minute*60+p_date.second)/60
df4['arr_time'] = (a_date.hour*3600+a_date.minute*60+a_date.second)/60
df4['Hour'] = a_date.hour
df4['Day'] = a_date.day

In [89]:
df4['diff'] = df4.pred_time - df4.arr_time

In [90]:
df4['diff_pos'] = df4['diff'].abs()

In [91]:
df5 = df4[df4.diff_pos < 100]
df5 = df5.dropna()

In [92]:
max(df5.diff_pos)

99.98333333333335

In [93]:
rmse = np.sqrt(mean_squared_error(df5['arr_time'], df5['pred_time']))
rmse

3.891478930486604

In [94]:
#df3.to_csv('merged_data.csv')